In [1]:
# Install Ultralytics if not installed
!pip install ultralytics --upgrade

import os
import shutil
import random
import yaml
import cv2
import glob
import numpy as np
from ultralytics import YOLO
from tqdm import tqdm
import albumentations as A

# Set random seed for reproducibility
random.seed(42)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.7/914.7 kB 14.9 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 2.0.3 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [2]:
def prepare_yolo_dataset(input_dir, output_dir, train_ratio=0.6, val_ratio=0.2, test_ratio=0.2, box_scale=1.0):
    """
    Converts a dataset into YOLO format by creating dummy bounding boxes and splitting into train/val/test.
    The bounding box size is dynamic, reducing over training phases to focus the model on relevant regions.
    """
    if os.path.exists(output_dir):
        shutil.rmtree(output_dir)
    for split in ['train', 'val', 'test']:
        os.makedirs(os.path.join(output_dir, split, 'images'), exist_ok=True)
        os.makedirs(os.path.join(output_dir, split, 'labels'), exist_ok=True)
    
    # Get sorted class names from subdirectories
    class_names = sorted([d for d in os.listdir(input_dir) if os.path.isdir(os.path.join(input_dir, d))])
    
    for cls in class_names:
        cls_dir = os.path.join(input_dir, cls)
        img_files = [f for f in os.listdir(cls_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        random.shuffle(img_files)
        n = len(img_files)
        n_train = int(n * train_ratio)
        n_val = int(n * val_ratio)
        train_imgs = img_files[:n_train]
        val_imgs = img_files[n_train:n_train+n_val]
        test_imgs = img_files[n_train+n_val:]

        for split, img_list in zip(['train', 'val', 'test'], [train_imgs, val_imgs, test_imgs]):
            for img in img_list:
                src = os.path.join(cls_dir, img)
                dst_img = os.path.join(output_dir, split, 'images', f"{cls}_{img}")
                shutil.copy(src, dst_img)
                base = os.path.splitext(os.path.basename(dst_img))[0]
                dst_label = os.path.join(output_dir, split, 'labels', f"{base}.txt")
                cls_id = class_names.index(cls)
                with open(dst_label, 'w') as f:
                    f.write(f"{cls_id} 0.5 0.5 {box_scale:.2f} {box_scale:.2f}\n")
    
    data_yaml = {
        'train': os.path.abspath(os.path.join(output_dir, 'train', 'images')),
        'val': os.path.abspath(os.path.join(output_dir, 'val', 'images')),
        'test': os.path.abspath(os.path.join(output_dir, 'test', 'images')),
        'nc': len(class_names),
        'names': class_names
    }
    yaml_path = os.path.join(output_dir, 'data.yaml')
    with open(yaml_path, 'w') as f:
        yaml.dump(data_yaml, f)
    print(f"✅ Prepared YOLO dataset in '{output_dir}' with {len(class_names)} classes and box_scale={box_scale:.2f}.")
    print(f"🔹 Data YAML file saved at: {yaml_path}")
    return yaml_path


In [3]:
def dynamic_yolo_training(dataset_input, base_output, model_file="yolov8x.pt",
                          initial_box_scale=1.0, final_box_scale=0.8, phases=3,
                          epochs_per_phase=10, imgsz=256):
    """
    Performs dynamic training using YOLO by progressively adjusting bounding box sizes.
    """
    box_scales = np.linspace(initial_box_scale, final_box_scale, phases).tolist()
    model = YOLO(model_file)

    for phase_idx, box_scale in enumerate(box_scales, 1):
        phase_output = os.path.join(base_output, f"phase_{phase_idx}")
        print(f"\n📌 Phase {phase_idx}/{phases}: Preparing dataset with box_scale = {box_scale:.2f}")
        yaml_file = prepare_yolo_dataset(dataset_input, phase_output, box_scale=box_scale)

        print(f"🚀 Starting training phase {phase_idx} with box_scale = {box_scale:.2f}...")
        model.train(data=yaml_file, epochs=epochs_per_phase, imgsz=imgsz, pretrained=(phase_idx == 1))
    
    return model


In [4]:
# Paths
plantvillage_input = "/kaggle/input/plant-data/raw/color"
plantvillage_output = "/kaggle/working/plantvillage_yolo"
thermal_input = "/kaggle/input/thermal-images-diseased-healthy-leaves-paddy/thermal images UL"
thermal_output = "/kaggle/working/thermal_yolo"

# Prepare datasets
plant_yaml = prepare_yolo_dataset(plantvillage_input, plantvillage_output, box_scale=1.0)
thermal_yaml = prepare_yolo_dataset(thermal_input, thermal_output, box_scale=1.0)


✅ Prepared YOLO dataset in '/kaggle/working/plantvillage_yolo' with 25 classes and box_scale=1.00.
🔹 Data YAML file saved at: /kaggle/working/plantvillage_yolo/data.yaml
✅ Prepared YOLO dataset in '/kaggle/working/thermal_yolo' with 6 classes and box_scale=1.00.
🔹 Data YAML file saved at: /kaggle/working/thermal_yolo/data.yaml


In [ ]:
thermal_dynamic_output_base = "/kaggle/working/thermal_yolo_dynamic"
trained_model = dynamic_yolo_training(
    thermal_input, thermal_dynamic_output_base,
    model_file="yolov11x.pt",  
    initial_box_scale=1.0, final_box_scale=0.8,
    phases=3, epochs_per_phase=10, imgsz=256
)


100%|██████████| 131M/131M [00:01<00:00, 85.7MB/s]



📌 Phase 1/3: Preparing dataset with box_scale = 1.00
✅ Prepared YOLO dataset in '/kaggle/working/thermal_yolo_dynamic/phase_1' with 6 classes and box_scale=1.00.
🔹 Data YAML file saved at: /kaggle/working/thermal_yolo_dynamic/phase_1/data.yaml
🚀 Starting training phase 1 with box_scale = 1.00...
Ultralytics 8.3.74 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8x.pt, data=/kaggle/working/thermal_yolo_dynamic/phase_1/data.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=256, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, io

100%|██████████| 755k/755k [00:00<00:00, 4.05MB/s]


Overriding model.yaml nc=80 with nc=6

                   from  n    params  module                                       arguments                     
  0                  -1  1      2320  ultralytics.nn.modules.conv.Conv             [3, 80, 3, 2]                 
  1                  -1  1    115520  ultralytics.nn.modules.conv.Conv             [80, 160, 3, 2]               
  2                  -1  3    436800  ultralytics.nn.modules.block.C2f             [160, 160, 3, True]           
  3                  -1  1    461440  ultralytics.nn.modules.conv.Conv             [160, 320, 3, 2]              
  4                  -1  6   3281920  ultralytics.nn.modules.block.C2f             [320, 320, 6, True]           
  5                  -1  1   1844480  ultralytics.nn.modules.conv.Conv             [320, 640, 3, 2]              
  6                  -1  6  13117440  ultralytics.nn.modules.block.C2f             [640, 640, 6, True]           
  7                  -1  1   3687680  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 17.1MB/s]


...


10 epochs completed in 0.028 hours.
Optimizer stripped from runs/detect/train22/weights/last.pt, 136.7MB
Optimizer stripped from runs/detect/train22/weights/best.pt, 136.7MB

Validating runs/detect/train22/weights/best.pt...
Ultralytics 8.3.74 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 268 layers, 68,129,346 parameters, 0 gradients, 257.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:01<00:00,  3.51it/s]


                   all        125        125      0.698       0.65      0.736      0.734
                   BLB         44         44      0.769      0.682      0.838      0.838
                 Blast         13         13       0.52      0.692      0.648      0.648
               healthy         18         18      0.686      0.889      0.917      0.917
                 hispa         28         28      0.741      0.679      0.818      0.818
           leaf folder          6          6      0.588      0.481      0.493      0.493
             leaf spot         16         16      0.884       0.48      0.705      0.693


/usr/local/lib/python3.10/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.10/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


Speed: 0.1ms preprocess, 5.1ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to runs/detect/train22


In [ ]:
# Standard Training on PlantVillage
print("\n🚀 Starting YOLOv11 training on PlantVillage dataset...")
model_pv = YOLO("yolov11x.pt")  # Ensure this file is available
model_pv.train(data=plant_yaml, epochs=10, imgsz=256)

# Standard Training on Thermal
print("\n🚀 Starting YOLOv11 training on Thermal dataset...")
model_th = YOLO("yolov11x.pt")
model_th.train(data=thermal_yaml, epochs=10, imgsz=256)



🚀 Starting YOLOv8 training on PlantVillage dataset...
Ultralytics 8.3.74 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8x.pt, data=/kaggle/working/plantvillage_yolo/data.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=256, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_

train: Scanning /kaggle/working/plantvillage_yolo/train/labels... 18141 images, 0 backgrounds, 0 corrupt: 100%|██████████| 18141/18141 [00:15<00:00, 1158.01it/s]


train: New cache created: /kaggle/working/plantvillage_yolo/train/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Scanning /kaggle/working/plantvillage_yolo/val/labels... 6042 images, 0 backgrounds, 0 corrupt: 100%|██████████| 6042/6042 [00:05<00:00, 1114.62it/s]


val: New cache created: /kaggle/working/plantvillage_yolo/val/labels.cache
Plotting labels to runs/detect/train3/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000345, momentum=0.9) with parameter groups 97 weight(decay=0.0), 104 weight(decay=0.0005), 103 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 256 train, 256 val
Using 2 dataloader workers
Logging results to runs/detect/train3
Starting training for 10 epochs...
Closing dataloader mosaic
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:01<00:00,  3.37it/s]


                   all        125        125      0.206      0.881      0.396      0.375
                   BLB         44         44      0.354          1      0.575      0.575
                 Blast         13         13      0.111      0.923      0.207      0.197
               healthy         18         18      0.162          1       0.65      0.538
                 hispa         28         28      0.371      0.893      0.572      0.572
           leaf folder          6          6     0.0997      0.667      0.164      0.164
             leaf spot         16         16      0.137      0.801      0.204      0.204


/usr/local/lib/python3.10/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.10/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


Speed: 0.1ms preprocess, 4.9ms inference, 0.0ms loss, 1.8ms postprocess per image
Results saved to runs/detect/train4


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7a4f6a8d3fa0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
     

In [ ]:
print("\n🔍 Evaluating YOLOv11 Model on PlantVillage...")
metrics_pv = model_pv.val()
print(metrics_pv)

print("\n🔍 Evaluating YOLOv11 Model on Thermal...")
metrics_th = model_th.val()
print(metrics_th)



🔍 Evaluating YOLOv8 Model on PlantVillage...
Ultralytics 8.3.74 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 268 layers, 68,147,643 parameters, 0 gradients, 257.5 GFLOPs


val: Scanning /kaggle/working/plantvillage_yolo/val/labels.cache... 6042 images, 0 backgrounds, 0 corrupt: 100%|██████████| 6042/6042 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 378/378 [01:48<00:00,  3.49it/s]


                   all       6042       6042      0.983      0.974      0.992       0.99
    Apple___Apple_scab        126        126      0.989      0.984      0.991      0.991
     Apple___Black_rot        124        124          1      0.996      0.995      0.992
Apple___Cedar_apple_rust         55         55      0.975          1      0.995      0.995
       Apple___healthy        329        329       0.95      0.988      0.993      0.991
   Blueberry___healthy        300        300      0.985          1      0.995      0.995
Cherry_(including_sour)___Powdery_mildew        210        210      0.995      0.979       0.99      0.989
Cherry_(including_sour)___healthy        170        170          1      0.973      0.994      0.991
Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot        102        102      0.925      0.852      0.955      0.946
Corn_(maize)___Common_rust_        238        238          1      0.983      0.994      0.993
Corn_(maize)___Northern_Leaf_Blight        197

/usr/local/lib/python3.10/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.10/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


Speed: 0.1ms preprocess, 14.9ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs/detect/train32
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7a4f14e7ef20>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032

val: Scanning /kaggle/working/thermal_yolo/val/labels.cache... 125 images, 0 backgrounds, 0 corrupt: 100%|██████████| 125/125 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:02<00:00,  3.83it/s]


                   all        125        125      0.206      0.878      0.395      0.374
                   BLB         44         44      0.354          1      0.579      0.579
                 Blast         13         13      0.113      0.923       0.21        0.2
               healthy         18         18      0.162          1       0.64      0.522
                 hispa         28         28      0.371      0.893      0.572      0.571
           leaf folder          6          6        0.1      0.667      0.164      0.164
             leaf spot         16         16      0.134      0.783      0.204      0.204


/usr/local/lib/python3.10/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.10/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


Speed: 1.0ms preprocess, 11.6ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs/detect/train42
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7a4f6a8e67a0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,   